In [70]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from scipy.spatial import cKDTree
from shapely.geometry import Point
from preparacion_datos.src.preparacion_inter_puntos import prep
import glob
import tqdm as tqdm
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.geometry

### PREDIAL

In [2]:
casas=gpd.read_file(r'C:\Users\dlara\Naucalpan_new_casas.shp')
# terrenos=gpd.read_file(r'C:\Users\dlara\Downloads\naucalpancompleto\new_terreno1.shp')

In [3]:
predial=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PREDIAL (16022022).xlsx')
adeudo=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PREDIALADEUDO (16022022).xlsx')
propietario=pd.read_excel(r'C:\Users\dlara\Downloads\Recepcion de informacion-20230302T185018Z-001\Recepcion de informacion\Naucalpan de Juárez_PROPIETARIO (16022022).xlsx')

In [4]:
base_pred= pd.read_csv(r'C:\Users\dlara\Downloads\Predial.csv')

In [9]:
from geoloc2.Valor_Catastral.calculo_predial import *

ImportError: attempted relative import with no known parent package

In [ ]:
curts= gpd.read_file(r'C:\Users\dlara\Downloads\098 Naucalpan\NAUCALPAN_PREDIOS.shp')

In [ ]:
NAUC=pd.read_csv(r'C:\Users\dlara\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv', encoding='utf-8-sig')

C:\Users\dlara\AppData\Local\Temp\ipykernel_14656\1341984383.py:1: DtypeWarning: Columns (2,4,5,6,8,9,10,11,12,13,14,15,16,17,19,20,21,23,24,25,26,27,30,32,34,35,38,39,40,43,44,49,50,51,52,53,55,56,57,60,61,62,63,64,65,66,67,68,69,70,72,73,74,75,76,78,79,80,81,82,83,84,85,86,87,88,89,96,97,98,99,101,102,103,104,107,108,109,110,111,113,114,115,116,117,118,119,124,127,129,130,131,134,135,137,138,139,140,141,142,146,156,157,158,159,160,161,162,164,166,167,168,169,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,219,220) have mixed types. Specify dtype option on import or set low_memory=False.
  NAUC=pd.read_csv(r'C:\Users\dlara\BASE_MEJORADA_NAUCALPAN_URLS_v2.csv', encoding='utf-8-sig')


In [ ]:
def transform_df_to_gpd(df: pd.DataFrame,
                    lon_col: str = 'INTFIS_RS_FD_LON_1',
                    lat_col: str = 'INTFIS_RS_FD_LAT_1', crs= 'EPSG:4326'):
        '''Transforma base a partir de dos columnas de latitud y longitud'''
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(
                df[lon_col], df[lat_col]
            ),
            crs=crs,
        )
        return gdf

In [ ]:
NAUC['INTFIS_RS_FD_LAT']= NAUC['INTFIS_RS_FD_LAT_1'].combine_first(NAUC['INTFIS_RS_FD_LAT'])
NAUC['INTFIS_RS_FD_LON']= NAUC['INTFIS_RS_FD_LON_1'].combine_first(NAUC['INTFIS_RS_FD_LON'])

In [ ]:
NAUC=transform_df_to_gpd(NAUC, lon_col= 'INTFIS_RS_FD_LON',lat_col = 'INTFIS_RS_FD_LAT')

In [ ]:
PRED= NAUC.loc[NAUC['CLAVECATASTRAL'].notna()]

In [ ]:
Manzanas=gpd.read_file(r'C:\Users\dlara\Downloads\Full manzanas\Full manzanas\Manzana_Naucalpan.shp')

In [ ]:
Manzanas['geometry']=Manzanas.buffer(100)

In [ ]:
Manz=Manzanas.dissolve()

In [ ]:
NAUC=NAUC.to_crs(Manz.crs)
def new_func(NAUC):
    return gpd.sjoin(NAUC,Manz).drop(columns=['index_right',	'mun',	'zona',	'manz',	'cve_cat'])

rectificado_nauc=new_func(NAUC)

In [ ]:
curts=curts.to_crs(Manz.crs)
curts_rectificado=gpd.sjoin(curts,Manz).drop(columns=['index_right',	'mun',	'zona',	'manz',	'cve_cat'])

In [ ]:
casas=casas.to_crs(Manz.crs)
casas['ID']=casas.reset_index()['index']
casas_rectificado=gpd.sjoin(casas,Manz).drop(columns=[	'index_right',	'mun',	'zona',	'manz',	'cve_cat'])

## Cruces

In [297]:
def line2poli(BASE):
    linestring=BASE.loc[(BASE["geometry"].map(lambda y: isinstance(y,MultiPolygon))==False)&(BASE["geometry"].map(lambda y: isinstance(y,Polygon))==False)].dissolve()
    try:
        ARR=linestring.geometry.astype(str)
        l1=ARR.str.find('LINE')
        for i in range(len(ARR)):
            list_wkt = ARR[i].replace(ARR[l1[i]-4:len(ARR[i])],'').replace("GEOMETRYCOLLECTION (",'')
            list_wkt= [list_wkt+str('))')]
            list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]

            multi=shapely.geometry.MultiPolygon(list_polygons)
            linestring['geometry'] = multi
            for col in dictt['cata']:
                BASE=BASE[~BASE[col].isin(linestring[col])]
            BASE = pd.concat([BASE,linestring])
    except:
        print("No linestring in base", print(BASE.head()))

        return(BASE)

In [96]:
linestring=shapes[name].loc[(shapes[name]["geometry"].map(lambda y: isinstance(y,MultiPolygon))==False)&(shapes[name]["geometry"].map(lambda y: isinstance(y,Polygon))==False)].dissolve()

ARR=linestring.geometry.astype(str)
l1=ARR.str.find('LINE')
for i in range(len(ARR)):
    list_wkt = ARR[i].replace(ARR[l1[i]-4:len(ARR[i])],'').replace("GEOMETRYCOLLECTION (",'')
    list_wkt= [list_wkt+str('))')]
    list_polygons =  [shapely.wkt.loads(poly) for poly in list_wkt]

    multi=shapely.geometry.MultiPolygon(list_polygons)
    linestring['geometry'] = multi
    for col in dictt['cata']:
        shapes[name]=shapes[name][shapes[name][col].isin(linestring[col])]
    shapes[name] = pd.concat([shapes[name],linestring])

In [258]:
dictt= {'cata':['id_cat','cve_cat','CLAVECATAS']}
dictt['cata']

['id_cat', 'cve_cat', 'CLAVECATAS']

In [23]:
for file in glob.iglob(path2 + '/*.shp'):
    file= file.replace('/',"\\")
    # gpd.read_file(file)
    pathh= pathh.replace('\\',"/")
    gpd.read_file(pathh)

In [24]:
gpd.read_file(pathh)

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry
0,15,024,0000,0240144801000000,194744919109952299131,None,"POLYGON ((-99.87493 19.79576, -99.87494 19.795..."
1,15,024,0000,0240109128000000,194737177109953100419,None,"POLYGON ((-99.88603 19.79363, -99.88612 19.793..."
2,15,024,0000,0240109116000000,194736032409953102406,None,"POLYGON ((-99.88603 19.79363, -99.88590 19.793..."
3,15,024,0000,0240142005000000,194824758809952102202,None,"POLYGON ((-99.86944 19.80691, -99.86948 19.806..."
4,15,024,0000,0240144837000000,194744232409952299340,None,"POLYGON ((-99.87493 19.79576, -99.87485 19.795..."
...,...,...,...,...,...,...,...
1919,15,024,0000,0240122320000000,194719536809952058052,None,"POLYGON ((-99.86814 19.78877, -99.86809 19.788..."
1920,15,024,0000,0240122321000000,194720252909952063672,None,"POLYGON ((-99.86820 19.78889, -99.86814 19.788..."
1921,15,024,0000,0240122328000000,194720252909952058206,None,"POLYGON ((-99.86827 19.78906, -99.86820 19.788..."
1922,15,024,0000,0240122345000000,194720239609952072994,None,"POLYGON ((-99.86859 19.78904, -99.86872 19.788..."


In [325]:

path=r"C:\Users\dlara\Desktop\MUNICIPIOS\Insumos_municipios_en_proceso\*"
filenames= glob.glob(path)
shapes={}
for files in tqdm.tqdm(filenames):
       
    i=0
    files=files.replace('\\',"/")
    path2=files.replace('\\',"/").replace('/*',"/")
    
    for file in glob.glob(path2 + '/*.shp'):
        file= file.replace('/',"\\")
        name=file.replace(path2.replace('/',"\\"),'').replace('.shp',"").replace('\\',"") 
        i=i+1
        Nname= name + str(i)
    #   
        shapes[Nname]=gpd.read_file(file)
        
        # print(Nname)
        # reemplazar id cat por clave catastral
        
        for col in dictt['cata']:
            shapes[Nname].rename(columns={col:'CLAVECATASTRAL'}, inplace=True)
        #Disolución de las geometrías a partir de clavecatastral 
        shapes[Nname]=shapes[Nname].dissolve(by='CLAVECATASTRAL').reset_index()
        
        if shapes[Nname].crs == 4326:
            pass
        else:
            shapes[Nname]=shapes[Nname].to_crs(4326)

    lista= list(shapes.keys())       
    for i in tqdm.tqdm(range(len(lista))):
        # if  (Nname.casefold() in i.casefold()):
        #     # print(Nname.casefold(), i.casefold())
        #     if Nname.casefold() == i.casefold():
                # print((Nname.casefold(), i.casefold()),)
                # continue
            # else:

        shps_merged= shapes[lista[i]]
        shps_merged.crs= 4326
        shps_merged=shapes[lista[i+1]].overlay(shps_merged, how='union',keep_geom_type=False)
        shps_merged= line2poli(shps_merged)
        
        shps_merged.to_file(path2+ Nname + '_cruce_shapes.shp')
        # else: 
            # print('error')  

  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\dlara\anaconda3\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\dlara\AppData\Local\Temp\ipykernel_15048\1306521757.py:45: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shps_merged.to_file(path2+ Nname + '_cruce_shapes.shp')
Normalized/laundered field name: 'CLAVECATASTRAL_1' to 'CLAVECATAS'
Normalized/laundered field name: 'CLAVECATASTRAL_2' to 'CLAVECAT_1'


   CLAVECATASTRAL_1  mun zona manz  CLAVECATASTRAL_2 cve_ent cve_mun cve_loc  \
0  0240100600000000  024   01  006  0240100601000000      15     024    0000   
1  0240100800000000  024   01  008  0240100822000000      15     024    0000   
2  0240100800000000  024   01  008  0240100827000000      15     024    0000   
3  0240100800000000  024   01  008  0240100828000000      15     024    0000   
4  0240101100000000  024   01  011  0240101114000000      15     024    0000   

                    curt notas  \
0  194804678009952240860  None   
1  194805341109952221128  None   
2  194805279409952217160  None   
3  194806038509952206157  None   
4  194814880409952222562  None   

                                            geometry  
0  POLYGON ((-99.87338 19.80148, -99.87320 19.801...  
1  POLYGON ((-99.87296 19.80133, -99.87291 19.801...  
2  POLYGON ((-99.87278 19.80143, -99.87276 19.801...  
3  POLYGON ((-99.87240 19.80161, -99.87245 19.801...  
4  POLYGON ((-99.87265 19.80406, -99.87

Attempt to write non-polygon (LINESTRING) geometry to POLYGON type shapefile.
  0%|          | 0/5 [00:03<?, ?it/s]


RuntimeError: GDAL Error: Attempt to write non-polygon (LINESTRING) geometry to POLYGON type shapefile. 
 
 Failed to write record: {'id': '2159', 'type': 'Feature', 'properties': {'CLAVECATASTRAL_1': '0240130500000000', 'CLAVECATASTRAL_2': None, 'curt': None, 'cve_ent': None, 'cve_loc': None, 'cve_mun': None, 'manz': '305', 'mun': '024', 'notas': None, 'zona': '01'}, 'geometry': {'type': 'GeometryCollection', 'geometries': [{'type': 'LineString', 'coordinates': ((-99.86865499131815, 19.787197678730617), (-99.86865862899998, 19.787198855000042))}, {'type': 'LineString', 'coordinates': ((-99.86922551415601, 19.78747242761317), (-99.86923730299998, 19.787439953000046))}, {'type': 'LineString', 'coordinates': ((-99.86952974199994, 19.787551955000026), (-99.86952372442653, 19.787568532714584))}, {'type': 'Polygon', 'coordinates': (((-99.87003552281242, 19.785937750890227), (-99.87025268499907, 19.78600684791602), (-99.87023205911505, 19.786072566005846), (-99.87007995899995, 19.78603618700

In [327]:
file

'C:\\Users\\dlara\\Desktop\\MUNICIPIOS\\Insumos_municipios_en_proceso\\Atlacomulco\\Manzana_Atlacomulco.shp'

In [311]:
shapes['ATLACOMULCO_PREDIOS1'].casefold()

{'ATLACOMULCO_PREDIOS1':         CLAVECATASTRAL                                           geometry  \
 0     0240100601000000  POLYGON ((-99.87319 19.80117, -99.87328 19.800...   
 1     0240100822000000  POLYGON ((-99.87272 19.80152, -99.87276 19.801...   
 2     0240100827000000  POLYGON ((-99.87275 19.80140, -99.87278 19.801...   
 3     0240100828000000  POLYGON ((-99.87234 19.80172, -99.87240 19.801...   
 4     0240101114000000  POLYGON ((-99.87265 19.80408, -99.87265 19.804...   
 ...                ...                                                ...   
 1881  0241530806000000  POLYGON ((-99.84686 19.80909, -99.84690 19.808...   
 1882  0241530815000000  POLYGON ((-99.84650 19.80933, -99.84652 19.809...   
 1883  0241530817000000  POLYGON ((-99.84680 19.80907, -99.84683 19.808...   
 1884  0241530822000000  POLYGON ((-99.84697 19.80856, -99.84700 19.808...   
 1885  0241530824000000  POLYGON ((-99.84688 19.80852, -99.84698 19.808...   
 
      cve_ent cve_mun cve_loc         

In [280]:
lista[i].casefold()
name.casefold()

'manzana_temoaya'

In [2]:
PRED=pd.read_csv(r'C:\Users\dlara\Atlacomulco_puntos.csv', encoding='utf-8-sig')
PRED['CLAVECATASTRAL']=PRED['CLAVECATASTRAL'].astype(str).str.replace('.0','', regex=False).str.zfill(16)
# 
PRED[['CLAVECATASTRAL', 'LONGITUD','LATITUD','CURT_1', 'curt','geometry','min_dist']].dropna()

# prep.transform_df_to_gpd(PRED, 'LONGITUD_1', 'LATITUD_1', 3857).to_crs(4326)

PRED.drop_duplicates('geometry', inplace=True)
PRED.dropna(axis=1, inplace=True)
puntos=prep.transform_df_to_gpd(PRED, 'LONGITUD_1', 'LATITUD_1', 3857)
puntos.dropna(axis=1, inplace=True)
rev_pred=gpd.read_file(r'C:\Users\dlara\Downloads\final\final\test_igecem_final.shp')

UNICOS=rev_pred.loc[rev_pred['CURT_1']==1]
nounicos=rev_pred.loc[rev_pred['CURT_1'].fillna(0)!=1]
nounicos=nounicos.to_crs(3857)
puntos=puntos.set_geometry('geometry')
rev_pred=rev_pred.set_geometry('geometry')
rev_pred.drop_duplicates('CLAVECATAS', keep='first', inplace=True)
# from preparacion_datos.src.preparacion_inter_puntos import ckdnearest

j_puntos=prep.ckdnearest(puntos, puntos[['geometry']])
cruce_check= gpd.sjoin(nounicos.to_crs(3857),j_puntos.loc[j_puntos['min_dist_2']>30][['geometry', 'LONGITUD_1', 'LATITUD_1','min_dist']], how='right')

cruce_check.to_file('C:/Users/dlara/Atlacomulco_predial_final_curts_reps.shp')
UNICOS.to_file('C:/Users/dlara/Atlacomulco_predial_final_curts_unicos.shp')

C:\Users\dlara\AppData\Local\Temp\ipykernel_10084\94833481.py:1: DtypeWarning: Columns (47,48,54) have mixed types. Specify dtype option on import or set low_memory=False.
  PRED=pd.read_csv(r'C:\Users\dlara\Atlacomulco_puntos.csv', encoding='utf-8-sig')


In [267]:
shapes.keys()

dict_keys(['ATLACOMULCO_PREDIOS1', 'Manzana_Atlacomulco2', 'Manzana_Ixtapan_Sal1', 'predios_0602', 'Manzana_Lerma1', 'prediosLerma2', 'Manzana_Naucalpan1', 'NAUCALPAN_PREDIOS2', 'Manzana_Temoaya1'])

In [268]:

filenames= glob.glob(r"C:\Users\dlara\Desktop\MUNICIPIOS\*.csv")

for path in tqdm.tqdm(filenames):
    filename= glob.glob(path)
    puntos=pd.read_csv(path)
    rev_pred=gpd.read_file(path[:path.find('_puntos.csv')] + '.shp')
    UNICOS=rev_pred.loc[rev_pred['CURT_1']==1]
    nounicos=rev_pred.loc[rev_pred['CURT_1'].fillna(0)!=1]
    nounicos=nounicos.to_crs(3857)
    puntos=puntos.set_geometry('geometry')
    rev_pred=rev_pred.set_geometry('geometry')
    rev_pred.drop_duplicates('CLAVECATAS', keep='first', inplace=True)
    j_puntos=prep.ckdnearest(puntos, puntos[['geometry']])
    cruce_check= gpd.sjoin(nounicos.to_crs(3857),j_puntos.loc[j_puntos['min_dist_2']>30][['geometry', 'LONGITUD_1', 'LATITUD_1','min_dist']], how='right')
#   cruce_check.loc[cruce_check['CLAVECATAS']=='0240127004000000'].plot()
    cruce_check.to_file(path[:path.find('_puntos.csv')] + '_predial_final_curts_reps.shp')
    UNICOS.to_file(path[:path.find('_puntos.csv')] + '_predial_final_curts_unicos.shp')


  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\dlara\AppData\Local\Temp\ipykernel_15048\1605995211.py:5: DtypeWarning: Columns (47,48,54) have mixed types. Specify dtype option on import or set low_memory=False.
  puntos=pd.read_csv(path)
C:\Users\dlara\Desktop\MUNICIPIOS\Atlacomulco.shp: No such file or directory
  0%|          | 0/2 [00:10<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "fiona\_shim.pyx", line 83, in fiona._shim.gdal_open_vector
  File "fiona\_err.pyx", line 291, in fiona._err.exc_wrap_pointer
fiona._err.CPLE_OpenFailedError: C:\Users\dlara\Desktop\MUNICIPIOS\Atlacomulco.shp: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\dlara\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\dlara\AppData\Local\Temp\ipykernel_15048\1605995211.py", line 6, in <module>
    rev_pred=gpd.read_file(path[:path.find('_puntos.csv')] + '.shp')
  File "c:\Users\dlara\anaconda3\lib\site-packages\geopandas\io\file.py", line 201, in _read_file
    with reader(path_or_bytes, **kwargs) as features:
  File "c:\Users\dlara\anaconda3\lib\site-packages\fiona\env.py", line 408, in wrapper
    return f(*args, **kwargs)
  F

In [ ]:
test0=pd.concat([m1,ADS_F,PRED_F, fin_casas], axis=0)
# .drop_duplicates('geometry')

In [ ]:
predial.loc[~predial['CLAVECATASTRAL'].isin(PRED['CLAVECATASTRAL'])]

### Distancias mínimas

In [ ]:
from shapely.geometry import Point, LineString
import geopandas as gpd
from geoloc2.preparacion_datos.src.preparacion_inter_puntos import ckdnearest
gdf=gpd.read_file("C:/Users/ASUS/Desktop/qgiszonas/points.shp")
gdf["LATLON"]=gdf["geometry"].x.astype("str")+","+gdf["geometry"].y.astype("str")
gdf


# usar primero sjoin_nearest y si hay mucho fallo usar la función ckdnearest

In [ ]:
casas=gpd.read_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\casas_t_especificios_1.shp")
casas=casas.to_crs(gdf.crs)
casas["LATLON"]=casas["geometry"].centroid.x.astype("str")+","+casas["geometry"].centroid.y.astype("str")
casas["geometry1"]=casas["geometry"]
casas["geometry"]=casas["geometry"].representative_point()
casas=casas.reset_index()
casas.rename(columns={"index":"FID"},inplace=True)
casas

In [ ]:
terreno=gpd.read_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\terreno_t_especificios_1.shp")
terreno=terreno.to_crs(gdf.crs)
terreno["LATLON"]=terreno["geometry"].centroid.x.astype("str")+","+terreno["geometry"].centroid.y.astype("str")
terreno["geometry1"]=terreno["geometry"]
terreno["geometry"]=terreno["geometry"].representative_point()
terreno

In [ ]:

# JFF_c = JFF_c.loc[JFF_c['min_dist'] < 10]
JFF_c

In [ ]:
JFF_terreno = ckdnearest(gdf.drop(columns='LATLON').to_crs(3857), terreno.to_crs(3857))
# JFF_terreno = JFF_terreno.loc[JFF_terreno['min_dist'] < 10]
JFF_terreno

In [ ]:
JFF_c["origen"]="Casas"
JFF_terreno["origen"]="Terreno"
concatenado=pd.concat([JFF_c,JFF_terreno],ignore_index=True)
concatenado.sort_values(by="min_dist",inplace=True)
concatenado=concatenado.drop_duplicates(subset=["CLAVECATAS"],keep="first")
concatenado

In [ ]:
concatenado[["FID","origen","CLAVECATAS","geometry"]].to_crs("3857").to_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\Puntos_cercanos.shp")


In [ ]:
a=concatenado[["FID","origen","CLAVECATAS","geometry1"]]
a=gpd.GeoDataFrame(a[["FID","origen","CLAVECATAS"]],geometry=a["geometry1"])
a.crs=6364
a=a.to_crs("3857")
a.to_file(r"C:\Users\ASUS\Desktop\qgiszonas\salida\Poligonos_cercanos.shp")